In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore, auth
from firebase_admin.firestore import FieldFilter
from google.cloud.firestore_v1.document import DocumentReference
from datetime import datetime
import json 
from dotenv import load_dotenv
from random import randint
import os
import re
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

load_dotenv()

cred = credentials.Certificate(os.getenv("firebase_secret"))
app = firebase_admin.initialize_app(cred)


db = firestore.client()


def load_json_file(filename):
    with open(filename) as fp:
        content = json.load(fp)
    return content


In [2]:
def process_reference(reference: DocumentReference):
    return reference.path

def process_array(array: list):
    return [process_reference(item) if type(item) == DocumentReference else item for item in array]

def process_dict(input_dict):
    output_dict = {}
    for key, value in input_dict.items():
        if type(value) == DocumentReference:
            output_dict[key] = process_reference(value)
        elif isinstance(value, dict):
            output_dict[key] = process_dict(value)
        elif isinstance(value, list):
            output_dict[key] = process_array(value)
        elif isinstance(value, datetime):
            output_dict[key] = value.isoformat()
        else:
            output_dict[key] = value
    return output_dict

# Itens

In [3]:
itens_stream = db.collection("items").stream()

itens = {} 

for item in itens_stream:
    itens[item.id] = process_dict(item.to_dict())

In [4]:
for key, val in itens.items():
    itens[key] = process_dict(val)


In [5]:
with open("json1123/itens.json", "w") as fp:
    json.dump(itens, fp, indent=2, ensure_ascii=False)

## Volumes

In [6]:
volumes_stream = db.collection("volumes").where(filter=FieldFilter("deleted", "==", False)).stream()

volumes = {}

for volume in volumes_stream:
    volumes[volume.id] = process_dict(volume.to_dict())


In [7]:
with open("json1123/volumes.json", "w") as fp:
    json.dump(volumes, fp, indent=2, ensure_ascii=False)